<a href="https://colab.research.google.com/github/samersonal/TAMIDS-Assessment/blob/main/RDASH_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
#Importing dependencies
import pandas as pd
import urllib3
import re
import requests

In [26]:
#Getting a list of all scholars in the directory of tamu@scholar
data = pd.read_csv('export.csv')

In [27]:
#Checking first 5 entries
data.head()

,First name,Last name,Preferred title,Overview,Position,Department,Individual
0,Darren,Hartl,Assistant Professor,NaN,Assistant Professor,Aerospace Engineering,https://scholars.library.tamu.edu/vivo/display...
1,Peter,Reilly,Professor,My research focuses generally in the area of A...,Professor of Law,School of Law,https://scholars.library.tamu.edu/vivo/display...
2,Ben,Welch,Clinical Professor,NaN,Clinical Professor,Mays Business School,https://scholars.library.tamu.edu/vivo/display...
3,Janice,Epstein,Instructional Associate Professor,NaN,Instructional Associate Professor,Mathematics,https://scholars.library.tamu.edu/vivo/display...
4,Angela,Mulcahy,Clinical Associate Professor,NaN,Clinical Associate Professor,School of Nursing,https://scholars.library.tamu.edu/vivo/display...


In [28]:
#Creating a list the list of Apis from where information will be extracted
apilist = []
url_pattern = r'https://api.library.tamu.edu/scholars-discovery/individual/\S+'

for i in data['Individual'][:100]: #Please remove [:100] to extract apis for all scholars in the directory. (Takes a lot of time to compute the list!!)
    url = i
    http = urllib3.PoolManager()
    resp = http.request('GET', url)
    result  = resp.data.decode('utf-8') #Getting the HTML code of the entire scholar profile
    apipre = re.findall(url_pattern, result) #Finding the api link
    apilink = apipre[0].split('?')[0]
    apilist.append(apilink) #Creating a list of all the api

In [29]:
#Extracting all publications for a scholar and returning a list
def publications(row):
    pub_list = []
    for i in range(len(row['publications'])):
        result = row['publications'][i]
        publication = result['label']
        pub_list.append(publication)

    return pub_list

In [30]:
#Creating an array of dictionaries containing required information of all the scholars
records = []
for i in range(len(apilist)):
    try:
        api_url = apilist[i]
        response = requests.get(api_url)
        data = response.json()
        record = {'Name': data['name'], 'Email' : data['primaryEmail'], 'Dept': data['organizations'][0], 'Keywords' : data['keywords'], 'Pub List': publications(data)}
        records.append(record)
    except:
        continue

#Creating a dataframe containg the required information of all scholars
scholar_dataset = pd.DataFrame(records)

In [13]:
scholar_dataset.head()

,Name,Email,Dept,Keywords,Pub List
0,"Hartl, Darren",darren.hartl@tamu.edu,Aerospace Engineering,"[Origami Engineering, Origami, Morphing Struct...",[An Uncoupled Method for Fluid-Structure Inter...
1,"Reilly, Peter",preilly@law.tamu.edu,School of Law,"[Win-win, Mba, Social Exchange Theory, Pedagog...",[Was Machiavelli Right? Lying in Negotiation a...
2,"Epstein, Janice",j-epstein@tamu.edu,Mathematics,"[5108 Quantum Physics, 51 Physical Sciences, M...",[Effective mass of 3He impurities in 4He films...
3,"Mulcahy, Angela",mulcahy@tamu.edu,School of Nursing,"[Nursing Students, Autogenic Training, Nursing...",[The Effect of Autogenic Training on Self-Effi...
4,"Liu, Yan",liu96@tamu.edu,Marketing,"[3506 Marketing, 35 Commerce, Management, Tour...",[More or Less? Consumer Goal Orientation and P...


In [20]:
#Converting the list to a string containing comma seperated items
def list_to_string(x):
  keyword_list = x
  return ', '.join(map(str, keyword_list))

In [21]:
scholar_dataset['Keywords'] = scholar_dataset['Keywords'].apply(list_to_string)
scholar_dataset['Pub List'] = scholar_dataset['Pub List'].apply(list_to_string)

In [22]:
scholar_dataset.head()

,Name,Email,Dept,Keywords,Pub List
0,"Hartl, Darren",darren.hartl@tamu.edu,Aerospace Engineering,"Origami Engineering, Origami, Morphing Structu...",An Uncoupled Method for Fluid-Structure Intera...
1,"Reilly, Peter",preilly@law.tamu.edu,School of Law,"Win-win, Mba, Social Exchange Theory, Pedagogy...",Was Machiavelli Right? Lying in Negotiation an...
2,"Epstein, Janice",j-epstein@tamu.edu,Mathematics,"5108 Quantum Physics, 51 Physical Sciences, Ma...",Effective mass of 3He impurities in 4He films....
3,"Mulcahy, Angela",mulcahy@tamu.edu,School of Nursing,"Nursing Students, Autogenic Training, Nursing ...",The Effect of Autogenic Training on Self-Effic...
4,"Liu, Yan",liu96@tamu.edu,Marketing,"3506 Marketing, 35 Commerce, Management, Touri...",More or Less? Consumer Goal Orientation and Pr...


In [31]:
#Exporting in an Excel
scholar_dataset.to_excel('Scholar_Info.xlsx', index = False)